## Importation de donnnes

In [40]:
data<-read.csv("data_visco.csv")
data_lasso <- read.csv("data_lasso.csv")
data_AIC <- read.csv("data_AIC.csv")


In [5]:
dim(data_lasso)
dim(data_AIC)

[1] 7247   29

[1] 7247   20

## t-IMSE standard

In [41]:
# Charger la librairie
library("KrigInv") #Pour t-IMSE
library("nloptr") #Pour la fonction d'optimisation Cobyla

In [6]:
#t-IMSE STANDARD APPLIQUEE AUX DONNEES 

x = data[1:60,1:116]
#x = matrix(sample(5:500, 116*30, replace=TRUE),nrow = 30)
data_reduced = data[1:120,1:116]
#des = data_reduced[c(-1)]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.00000001,max = 0.00000002), nrow = 120)
mod = km(formula = ~1, design = data_reduced + noise, response = data_reduced[1])
tmse_optim(x, model = mod, T = 400)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  12076 0.78 1.976293e-08 44.86 1.95

[1] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
 [9] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[17] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[25] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[33] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[41] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[49] 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031 341.8031
[57] 341.8031 341.8031 341.8031 341.8031

## Adaptation t-IMSE à nos contraintes

Au lieu d'utiliser la fonction **tmse_optim** standard, il nous faut considerer les contraintes du simplex

In [42]:
# Initialize randomly data by index
d = 29
n_init = d*10
index = sample(1:nrow(data_lasso),n_init)
data_lasso_reduced = data_lasso[index, ]

X = data_lasso_reduced[,c(-1)]

In [43]:
#simulation de x unifome sur [0,1] et la somme egale à 1
simul <- function(d)
{
  # set.seed(18)
  u = runif(n=d-1, min=0,max=1)
  #replicate(n=5,u) si on doit tester avec plusieurs vecteurs diff
  u = c(0,u,1)
  u = sort(u)
  x = u[2:(d+1)] - u[1:d]  
  return(x)
}

In [44]:
g = function (x, model, T, method.param = NULL) 
{
    y <- t(x)
    if (ncol(y) == model@d) 
        z <- y
    if (ncol(y) != model@d) 
        z <- x
    krig <- predict_nobias_km(object = model, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    mk <- krig$mean
    sk <- krig$sd
    if (is.null(method.param)) 
        method.param <- 0
    epsilon <- method.param
    if (length(T) == 1) {
        W <- 1/sqrt(2 * pi * (sk^2 + epsilon^2)) * exp(-0.5 * 
            ((mk - T)/sqrt(sk^2 + epsilon^2))^2)
    }
    else {
        W0 <- 1/sqrt(2 * pi * (sk^2 + epsilon^2))
        W <- 0
        for (i in 1:length(T)) {
            Ti <- T[i]
            W <- W + W0 * exp(-0.5 * ((mk - Ti)/sqrt(sk^2 + epsilon^2))^2)
        }
    }
    g <- W * sk^2
    return(g)
}

In [45]:
g_new = function (x, m0, model, T, method.param = NULL) 
{
    y <- t(x)
    if (ncol(y) == model@d) 
        z <- y
    if (ncol(y) != model@d) 
        z <- x
    krig0 <- predict_nobias_km(object = m0, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    krig <- predict_nobias_km(object = model, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    mk0 <- krig0$mean
    sk0 <- krig0$sd
    if (is.null(method.param)) 
        method.param <- 0
    epsilon <- method.param
    if (length(T) == 1) {
        W <- 1/sqrt(2 * pi * (sk0^2 + epsilon^2)) * exp(-0.5 * 
            ((mk0 - T)/sqrt(sk0^2 + epsilon^2))^2)
    }
    else {
        W0 <- 1/sqrt(2 * pi * (sk0^2 + epsilon^2))
        W <- 0
        for (i in 1:length(T)) {
            Ti <- T[i]
            W <- W + W0 * exp(-0.5 * ((mk0 - Ti)/sqrt(sk0^2 + epsilon^2))^2)
        }
    }
    sk <- krig$sd
    g <- W * sk^2
    return(g)
}

In [46]:
tmse <- function(d,mod, T)
{
    s = 0
    for(i in 1:1000){
    x = simul(d) 
    s = s + g(x, model = mod,T)
    }
    tmse = s/1000
    return(tmse)
}
    

In [47]:
#Pour comparer avec la methode de Monte Carlo
#Calculer les resultats par t-IMSE standard
d = 29
N = 10*d
s = 0
data_Lasso_reduced = data_lasso[1:N,]
shape = c(dim(data_Lasso_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = N)
mod = km(formula = ~1, design = data_Lasso_reduced+noise, response = data_Lasso_reduced$vis)
x = data_Lasso_reduced[1:50,]
tmse_optim(x, model = mod, T = 400)



optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  476715169234 0.0001993814 44.86005 149.5402 0.9201835 0.0001985878 22.92014 0.6001868 0.00019927 0.0001993187 0.0001978581 0.08002992 105.1001 0.0001991955 57.84017 16.82009 3.000166 0.0001991671 110.7202 0.0001996859 2.000174 0.0001989328 141.6 0.0001988699 0.6201993 0.0001996406 0.0001997634 10.0002 157.6199 
  - best initial criterion value(s) :  -7132.313 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7132.3  |proj g|=       2.7067
At iterate     1  f =       6920.4  |proj g

[1] 538173507 538063484 536998161 528407485 537240893 539723642 537392231
 [8] 537968331 533084284 540040634 539482301 537537888 538714848 526245683
[15] 519543741 535969177 533581173 539147783 537242809 535008517 535008517
[22] 532670628 532670628 532886634 534142110 534154121 533072852 535256345
[29] 533784475 533273879 533273879 536520158 521963793 526515095 519749037
[36] 520250386 526456331 522878582 528684947 521595749 532141246 520896931
[43] 532991925 529930787 532314649 534154121 532384990 534176089 533671683
[50] 533671683

In [48]:
#Resultats par la methode Monte Carlo 
print(tmse(d,mod,T=400))

[1] 461468117


### Optimisation

In [49]:
#Modele initial
noise = matrix(runif(prod(dim(X)),min = 0.0001,max = 0.0002), nrow = nrow(X))
m0 = km(formula = ~1, design = X + noise, response = data_lasso_reduced[1],)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  0.0001985707 69.72006 54.06018 91.60009 50.00018 100.0001 21.7201 4.25018 119.7201 0.02011788 0.160183 68.64017 59.08 43.64002 36.40012 95.7001 0.0001977153 82.90007 0.0001989361 12.28005 9.380183 160 1.000041 1.100138 29.24016 5.880081 20.00001 155.54 
  - best initial criterion value(s) :  -7447.801 

N = 28, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7447.8  |proj g|=       4.0758
At iterate     1  f =       7362.8  |proj g|=        2.5342
At iterate     2  f =       7359.5  |proj g|=   

In [50]:
#TROUVER LE POINT LE PLUS PROCHE D'UN NOUVEAU POINT

#input: nouveau point: 'x_new' ; donnees dans lesquelles on cherche: 'data'
#output: la valeur y du point le plus proche
pointProche = function(x_new, data){
    X_cher = data[,c(-1)]
    argmin = NULL 
    min = norm(x_new- X_cher[1,], type = "2") 
    n_points = nrow(data) #longeur da la base de données

    for (i in range(n_points)){
        mse = norm(x_new- X_cher[i,], type = "2")
        if (mse < min){
            min = mse 
            argmin = i
        }
    }
    return (data[i,1])
}


In [14]:
library("KrigInv")

In [51]:

new_tmse = function(x_new){
    # d: dimension, d = 29
    data = data_updated
    d = 29
    niter = 1000
    s = 0
    
    y_new = pointProche(x_new, data)
    vis = y_new
    newpoint = c(vis,x_new)
    data_new = data.frame(data)
    data_new[nrow(data_new) + 1, ] = newpoint
    # data_new = rbind(data_new, as.numeric(newpoint))
    noise = matrix(runif(prod(dim(data_new)),min = 0.0001,max = 0.0002), nrow = nrow(data_new))
    #mod = km(formula = ~1, design = data_new + noise, response = data_new[1],)
    mod = update(m0, newX = data_new[,c(-1)]+noise, newy = data_new[,1],newX.alreadyExist = FALSE, cov.reestim = FALSE, trend.reestim = FALSE)
    for(i in 1:niter){
        x = simul(d-1)
        s = s + g_new(x, m0 = m0, model = mod, T = 400)
    }
    tmse = s/niter
    return(tmse)    
}

In [55]:
contraintes = function(x){
  n = length(x)
  h<-numeric(n+2)
  h[1]<- 100 - sum(x) #somme = 1
  h[2]<- sum(x) - 100
  #h[3:n+2] = as.numeric(x)
  for (i in range(3,n+2)){
    h[i] = x[i-2]
  }
  return(h)
}

In [56]:
contraintes(as.numeric(X[1,]))

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3

In [45]:
new_tmse(x_new = as.numeric(X[1,]))

[1] 39374.85

In [54]:
data_updated = data_lasso_reduced
cobyla(x0 = as.numeric(X[1,]),fn = new_tmse, hin = contraintes)

For consistency with the rest of the package the inequality sign may be switched from >= to <= in a future nloptr version.



ERROR: Error in is.nloptr(ret): inequality constraints in x0 returns NA


In [57]:
# OPTIMISATION TMSE PROCESS USING COBYLA

# Some parameters
d = 29  # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_lasso) # total number of data points
n_adds = 2 # number of additional data points

data_updated = data_lasso_reduced

print("Starting...")
# Loop over additional data points
for (i in 1:n_adds){

    print("Adding point... ")
    x_new = cobyla(x0 = as.numeric(X[i,]), fn = new_tmse, hin = contraintes, control = list(xtol_rel = 1e-3))$par
    y_new = pointProche(x_new,data_lasso)
    vis = y_new
    newpoint = cbind(vis,x_new)
    data_updated[nrow(data_updated) + 1, ] = newpoint
    #data_updated = rbind(data_updated,newpoint)
    X = data_updated[,c(-1)]
}

[1] "Starting..."
[1] "Adding point... "


For consistency with the rest of the package the inequality sign may be switched from >= to <= in a future nloptr version.

Warning message in matrix(value, n, p):
"data length [56] is not a sub-multiple or multiple of the number of columns [29]"


[1] "Adding point... "


For consistency with the rest of the package the inequality sign may be switched from >= to <= in a future nloptr version.

Warning message in matrix(value, n, p):
"data length [56] is not a sub-multiple or multiple of the number of columns [29]"


In [58]:
X 

,X__3,X__4,X__8,X__10,X__17,X__18,X__29,X__30,X__43,X__48,...,X__84,X__86,X__106,X__109,X__113,X__115,X__117,X__133,X__138,autres
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1561,0,12.000000,2.000000,0.0000000,0,12.0000000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,61.00000,0.00,0.0000000,10.000000,0.00,0.000000,3.00000
5446,0,16.980000,10.990000,5.4900000,0,4.9900000,0.0000000,0.00000000,0.00,0.00999,...,0,0.0000000,0,57.46000,0.00,0.0000000,1.990000,0.00,0.000000,1.10001
5206,0,13.300000,0.000000,0.0000000,0,0.0000000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,68.60000,0.00,0.0000000,0.000000,0.00,0.000000,2.60000
6035,0,9.500000,3.000000,0.0000000,0,0.0000000,0.0000000,0.00000000,0.00,0.00000,...,0,1.0000000,0,58.10000,0.00,0.0000000,0.000000,0.00,2.000000,2.40000
2179,0,10.860000,0.000000,2.3000000,0,12.7800000,0.0000000,0.00000000,23.38,0.00000,...,0,0.0000000,0,50.17000,0.00,0.0000000,0.000000,0.00,0.000000,0.51000
6790,0,10.700000,0.000000,0.0000000,0,6.5000000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,65.60000,0.00,0.0000000,0.000000,0.00,0.300000,0.00000
3247,0,0.500000,0.000000,0.0000000,0,11.0000000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,70.00000,0.00,0.0000000,4.000000,0.00,3.000000,0.00000
4991,0,5.600000,0.000000,11.4000000,0,1.4000000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,53.80000,0.00,0.0000000,4.300000,0.00,0.000000,10.20000
6484,0,6.090000,5.870000,0.0000000,0,7.1900000,0.0000000,0.00000000,0.00,0.00000,...,0,0.0000000,0,64.05000,0.03,0.2900000,0.160000,0.00,0.000000,0.04000


In [ ]:
# OPTIMISATION TMSE PROCESS

# Some parameters
d = 29  # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_lasso) # total number of data points
n_adds = 2 # number of additional data points

# Initialize randomly data by index
index = sample(1:nrow(data_lasso),n_init)
X = data_lasso[index, ]

print("Starting...")
# Loop over additional data points
for (i in 1:n_adds){
    # Loop over all data points except the data that are already selected
    print(i)
    argmin = NULL
    tmse = new_tmse(x_new = NULL, data = X)
    for (k in setdiff(1:n_points, index)){
        print("Loop at k = ", k)
        new_tmse = new_tmse(x_new = data_lasso[k,], data = X, niter = 100)
        if (new_tmse < tmse) argmin = k
        
    }
    print(argmin)
    X = rbind(X, data_lasso[argmin,])
    index = c(index, argmin)
}

[1] "Starting..."


In [ ]:
# OPTIMISATION TMSE PROCESS

# Some parameters
d = 29  # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_lasso) # total number of data points
n_adds = 2 # number of additional data points

# Initialize randomly data by index
index = sample(1:nrow(data_lasso),n_init)
X = data_lasso[index, ]

print("Starting...")

# Loop over additional data points
for (i in 1:n_adds){
    # Loop over all data points except the data that are already selected
    print(i)
    argmin = NULL
    tmse = new_tmse(x_new = NULL, data = X)
    for (k in setdiff(1:n_points, index)){
        print("Loop at k = ", k)
        new_tmse = new_tmse(x_new = data_lasso[k,], data = X, niter = 100)
        if (new_tmse < tmse) argmin = k
        
    }
    print(argmin)
    X = rbind(X, data_lasso[argmin,])
    index = c(index, argmin)
}

[1] "Starting..."

optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  1.907611e+12 0.0001986403 80.00001 73.56002 90.00014 74.26018 89.48016 15.56011 0.5381778 115.2401 67.98002 1.760002 51.66008 77.48011 39.30007 39.5601 54.60006 0.0001982339 79.50011 0.0001978923 12.08003 4.280067 158.9799 3.4001 1.000123 59.80001 28.00015 19.20201 157.6199 
  - best initial criterion value(s) :  -7314.788 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7314.8  |proj g|=       2.1492
At iterate     1  f =       7106.4  |proj g|=        2.6348
At

In [ ]:
print(tmse)

[1] 1964788529


## Changement de variables

### En coordonnées sphériques

In [63]:
library("SphericalCubature")

In [66]:
polar_AIC = rect2polar(data_AIC)
polar_lasso = rect2polar(data_lasso)
polar_full = rect2polar(data)

### En coordonnées...

In [ ]:
#CHANGEMENT DE VARIABLES POUR UN LIGNE

change_line = function(x){
    n = length(x)
    lambda = numeric(n-1)
}